## Generate Interpolations from Stored Vectors

Generating interpolation videos from POM2021 performance. 

Based on [j.mp/wanderclip](https://j.mp/wanderclip) by Eyal Gruss [@eyaler](https://twitter.com/eyaler) [eyalgruss.com](https://eyalgruss.com)

Modified to run on HCC OOD/nautilus.optiputer.net/z8 by robert.twomey@gmail.com

# 1. Setup and Imports

Setup the timer in case we want it

In [1]:
!pip install ipython-autotime

Defaulting to user installation because normal site-packages is not writeable


In [2]:
%load_ext autotime

time: 187 µs (started: 2021-12-21 12:17:49 -06:00)


In [3]:
!module load ffmpeg

time: 635 ms (started: 2021-12-21 12:17:49 -06:00)


Onetime setup (jupyter environment)
(you may need to restart the kernel after this)

In [4]:
# # !pip install perlin-noise
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install pytorch-pretrained-biggan
!pip install nltk
!pip install cma

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
time: 4.31 s (started: 2021-12-20 12:58:34 -06:00)


Import necessary libraries

In [12]:
from IPython.display import HTML, clear_output
from PIL import Image
from IPython.display import Image as JupImage
import numpy as np
import nltk
from scipy.stats import truncnorm

# from biggan
import torch
from pytorch_pretrained_biggan import (BigGAN, one_hot_from_names, truncated_noise_sample,
                                       save_as_images, convert_to_images) #, display_in_terminal)
import logging
logging.basicConfig(level=logging.WARNING)

# do we need wordnet?
nltk.download('wordnet')

# load biggan
model = BigGAN.from_pretrained('biggan-deep-512')
print("loaded bigGAN")

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/twomeylab/rtwomey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


loaded bigGAN
time: 3.44 s (started: 2021-12-21 12:18:20 -06:00)


More imports for interpolation

In [13]:
from numpy import asarray
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from numpy import arccos
from numpy import clip
from numpy import dot
from numpy import sin
from numpy import linspace
from numpy.linalg import norm
import os
import glob

time: 505 µs (started: 2021-12-21 12:18:24 -06:00)


Helper functions for interpolation

In [14]:
# from
# https://discuss.pytorch.org/t/help-regarding-slerp-function-for-generative-model-sampling/32475/4

# spherical linear interpolation (slerp)
def slerp(val, low, high):
    omega = arccos(clip(dot(low/norm(low), high/norm(high)), -1, 1))
    so = sin(omega)
    if so == 0:
        # L'Hopital's rule/LERP
        return (1.0-val) * low + val * high
    return sin((1.0-val)*omega) / so * low + sin(val*omega) / so * high
 
# uniform interpolation between two points in latent space
def interpolate_points(p1, p2, n_steps=10):
    # interpolate ratios between the points
    ratios = np.linspace(0, 1, num=n_steps)
    # linear interpolate vectors
    vectors = list()
    for ratio in ratios:
        v = slerp(ratio, p1, p2)
        vectors.append(v)
    return np.asarray(vectors)

time: 693 µs (started: 2021-12-21 12:18:27 -06:00)


File output settings

In [62]:
import os
# file paths
work = os.getcwd() # get the current path

# No need to seed the interpolation for generation from stored vectors
# np.random.RandomState(1)
# np.random.seed(1)
# torch.manual_seed(1)

# one random variable
truncation = 1.0

# the file directories
workbase = work

# results of text to image translation
resultspath = os.path.join(work, "results/")

# intermediate frames working directory
interpbase = os.path.join(work, "interpolation/")

# video directory
videopath = os.path.join(work, "videos/")

# video and srt output files
# filebase = '{0}'
# moviefilename = filebase+'.mp4'
# srtfilename = filebase+'.srt'

# the interpolation
num_steps = 90#300
len_hold = 30

# frame rate for the the movie
fps = 30

time: 962 µs (started: 2021-12-21 15:04:30 -06:00)


In [63]:
!mkdir -p $interpbase
!mkdir -p $videopath

time: 300 ms (started: 2021-12-21 15:04:31 -06:00)


## 2. Generate Interpolation Between Two Images

In [64]:
def frames_to_TC (frames):
    h = int(frames / 86400) 
    m = int(frames / (60*fps)) % 60 
    s = int((frames % (60*fps))/fps) 
    f = float(frames % (60*fps) % fps)/float(fps)
    return ( "%02d:%02d:%02d,%04d" % ( h, m, s, f))

time: 560 µs (started: 2021-12-21 15:05:18 -06:00)


In [65]:
def make_safe_filename(s):
    def safe_char(c):
        if c.isalnum():
            return c
        else:
            return "_"
    return "".join(safe_char(c) for c in s).rstrip("_")

time: 371 µs (started: 2021-12-21 15:05:18 -06:00)


In [70]:
def generate_video_transition(class_filenames, prompts, interpbase):
    """includes the result with the class_files"""
    
    print("==== retrieve vectors ====")    
    safe_prompt = make_safe_filename(prompts[0])
    noise_filenames = [fname.replace("class", "noise") for fname in class_filenames]

    #print(class_filenames)
    #print(noise_filenames)

    class_inputs = [np.loadtxt(filename) for filename in class_filenames]
    noise_inputs = [np.loadtxt(filename) for filename in noise_filenames]

    # file outputs names and paths
    moviefilename = safe_prompt+'_{0}.mp4'.format(fps)
    srtfilename = safe_prompt+'_{0}.srt'.format(fps)
    interppath = os.path.join(interpbase, safe_prompt)

    !mkdir -p $interppath
    
    print("==== generate interpolations ====")

    count = 0

    for i in range(len(class_inputs)):

        # generate interpolations
        noises = interpolate_points(noise_inputs[i], noise_inputs[(i+1)%len(class_inputs)], num_steps)
        classes = interpolate_points(class_inputs[i], class_inputs[(i+1)%len(class_inputs)], num_steps)

        # generate images in batches
#         batch_size = 10
        batch_size = 40 #RTX8000
    
        for j in range(0, num_steps, batch_size):
            clear_output()
            print(i, j, count)
            noise_vector = noises[j:j+batch_size]
            class_vector = classes[j:j+batch_size]

            # convert to tensors
            noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
            class_vector = torch.tensor(class_vector, dtype=torch.float32)

            # put everything on cuda (GPU)
            noise_vector = noise_vector.to('cuda')
            noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
            class_vector = class_vector.to('cuda')
            class_vector = class_vector.softmax(dim=-1)
            model.to('cuda')

            # generate images
            with torch.no_grad():
                output = model(noise_vector, class_vector, truncation)

            # If you have a GPU put back on CPU
            output = output.to('cpu')

            imgs = convert_to_images(output)

            # repeat first image for len_hold
            if j == 0:
                for k in range(len_hold):
                    imgs[0].save(interppath+"/output_%05d.png" % count)
                    count = count + 1

            for img in imgs: 
                img.save(interppath+"/output_%05d.png" % count)
                count = count + 1
    
    
    print("==== generating movie ====")
    
    # generate mp4
    outpath = os.path.join(videopath, moviefilename)
    with open('list.txt','w') as f:
      for i in range(count):
        f.write('file %s/output_%05d.png\n'%(interppath, i))
    cmd = "ffmpeg -r {0} -i {1}/output_%05d.png -c:v libx265 -pix_fmt yuv420p -crf 0 -r {0} {2} -y"
    
    print("video creation cmd:\n", cmd.format(fps, interppath, outpath))
#     os.system(cmd.format(fps, interppath, outpath))
    
    print("==== generating subtitles ==== ")
    srtoutpath = os.path.join(videopath, srtfilename)
    with open(srtoutpath,'w') as f:

        frames = 0
        for i in range(len(class_inputs)):
            f.write(str(i+1)+"\n")
            f.write(frames_to_TC(frames)+" --> ")
            frames+=len_hold
            f.write(frames_to_TC(frames)+"\n")
            f.write(prompts[i]+"\n\n")
            frames+=num_steps
    print(frames)

time: 3.15 ms (started: 2021-12-21 15:18:43 -06:00)


In [71]:
prompts = [
    "a drawing of an elegant machine",
    "a machine that learns to make images"
]

latentfiles = [
    "a_drawing_of_an_elegant_machine_class.txt",
    "a_machine_that_learns_to_make_images_class.txt"
]

class_filenames = [resultspath+latent for latent in latentfiles]
    
generate_video_transition(class_filenames, prompts, interpbase)

1 80 230
==== generating movie ====
video creation cmd:
 ffmpeg -r 30 -i /home/twomeylab/rtwomey/cohab-ml-imagination/interpolation/a_drawing_of_an_elegant_machine/output_%05d.png -c:v libx265 -pix_fmt yuv420p -crf 0 -r 30 /home/twomeylab/rtwomey/cohab-ml-imagination/videos/a_drawing_of_an_elegant_machine_30.mp4 -y
==== generating subtitles ==== 
240
time: 31.4 s (started: 2021-12-21 15:18:43 -06:00)


In [ ]:
# # move to datestamped path
# import os, datetime
# newdir = outpath[:-1]+"_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
# !mv $outpath $newdir
# !mkdir -p $outpath

In [ ]:
# from IPython.display import display, FileLink, FileLinks

# local_file = FileLink(out.replace('"', ""), result_html_prefix="Click here to download: ")
# # local_file = FileLinks(".", result_html_suffix="?download")
# display(local_file)

In [ ]:
# from IPython.display import HTML
# local_file = out.replace('"', "")
# HTML("<a href=\""+local_file+"\">download %s</a>"%local_file)

## 3. Generate Video Interpolating Between Multiple VHolding on Single Phrase

Helper for generating time codes for subtitles:

In [72]:
def frames_to_TC (frames):
    h = int(frames / 86400) 
    m = int(frames / (60*fps)) % 60 
    s = int((frames % (60*fps))/fps) 
    f = float(frames % (60*fps) % fps)/float(fps)
    return ( "%02d:%02d:%02d,%04d" % ( h, m, s, f))

time: 511 µs (started: 2021-12-21 15:29:11 -06:00)


In [73]:
def make_safe_filename(s):
    def safe_char(c):
        if c.isalnum():
            return c
        else:
            return "_"
    return "".join(safe_char(c) for c in s).rstrip("_")

time: 341 µs (started: 2021-12-21 15:29:12 -06:00)


In [ ]:
def generate_video_hold(output_folder, class_filenames, result_filename, prompt, interpbase, filebase):
    """includes the result with the class_files"""
    
    print("==== retrieve vectors ====")
    
    safe_prompt = make_safe_filename(prompt)

    prompts = [ prompt*(len(class_filenames)+1)]

    class_filenames = [output_folder+file for file in class_filenames]
    class_filenames += [result_filename.format(safe_prompt)]

    noise_filenames = [fname.replace("class", "noise") for fname in class_filenames]

    print(class_filenames)
    print(noise_filenames)

    class_inputs = [np.loadtxt(filename) for filename in class_filenames]
    noise_inputs = [np.loadtxt(filename) for filename in noise_filenames]


    interpbase = interpbase.format(safe_prompt)
    filebase = filebase.format(safe_prompt)
    moviefilename = filebase+'.mp4'
    srtfilename = filebase+'.srt'

    !mkdir -p $interpbase
    
    print("==== generate interpolations ====")

    count = 0
    
    num_inputs = len(class_inputs)
    for i in range(len(class_inputs)):

        # generate interpolations
        noises = interpolate_points(noise_inputs[i], noise_inputs[(i+1)%len(class_inputs)], num_steps)
        classes = interpolate_points(class_inputs[i], class_inputs[(i+1)%len(class_inputs)], num_steps)

        # generate images in batches
#         batch_size = 10 #2080
        batch_size = 60 #RTX8000
        for j in range(0, num_steps, batch_size):
            clear_output()
            print("{0}/{1} clases\t{2}/{3} steps\t{4} frames".format(i, num_inputs, j, num_steps, count))
            noise_vector = noises[j:j+batch_size]
            class_vector = classes[j:j+batch_size]

            # convert to tensors
            noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
            class_vector = torch.tensor(class_vector, dtype=torch.float32)

            # put everything on cuda (GPU)
            noise_vector = noise_vector.to('cuda')
            noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
            class_vector = class_vector.to('cuda')
            class_vector = class_vector.softmax(dim=-1)
            model.to('cuda')

            # generate images
            with torch.no_grad():
                output = model(noise_vector, class_vector, truncation)

            # If you have a GPU put back on CPU
            output = output.to('cpu')

            imgs = convert_to_images(output)

            # repeat first image

            if j == 0:
                for k in range(len_hold):
                    imgs[0].save(interpbase+"/output_%05d.png" % count)
                    count = count + 1

            for img in imgs: 
                img.save(interpbase+"/output_%05d.png" % count)
                count = count + 1
    
    
    print("==== generating movie ====")
    
    # generate mp4
    out = moviefilename%fps
    with open('list.txt','w') as f:
      for i in range(count):
        f.write('file %s/output_%05d.png\n'%(interpbase, i))
    cmd = "ffmpeg -r {0} -i {1}/output_%05d.png -c:v libx265 -pix_fmt yuv420p -crf 0 -r {0} {2} -y"

    os.system(cmd.format(fps, interpbase, out))
    print(cmd.format(fps, interpbase, out))
    
    print("==== generating subtitles ==== ")
    
    with open(srtfilename%fps,'w') as f:

        frames = 0
        for i in range(len(class_inputs)):
            f.write(str(i+1)+"\n")
            f.write(frames_to_TC(frames)+" --> ")
            frames+=len_hold
            f.write(frames_to_TC(frames)+"\n")
            f.write(prompt+"\n\n")
            frames+=num_steps
    print("subtitles: "+srtfilename%fps)
    print(frames)

In [ ]:
# resultsbase = "/home/jovyan/work/results/results_pom_seed_128/"

# # video and srt output files
# filebase = 'pom_128_%s'
# # filebase = 'pom_10_%s'
# moviefilename = filebase+'.mp4'
# srtfilename = filebase+'.srt'

# prompts = [
# #     "the sublime experience of an iceberg",
# #     "ice is a vastness of possibilities",
# #     "a vastness of spatial dimensions",
# #     "a space of unlimited possibilities that the network must explore",
# #     "a network",
# #     "a network of vast spatial dimensions",
# #     "eighteenth century painting of humans encountering nature",
# #     "a person encountering nature",
# #     "an elegant machine that learns to generate artificial images",
# #     "a drawing of an elegant machine",
# #     "a machine that learns to make images",
#     "a drawing of a machine that learns to make images",
# ]

# for prompt in prompts:
    
#     # prompt = "a silicon mold of a friends hands"

#     safe_prompt = make_safe_filename(prompt)

#     class_filenames = [ 
#         "class_00009.txt", 
#     #     "class_00008.txt", 
#         "class_00007.txt"
#     ]

#     result_filename = resultsbase+"{0}_class.txt"

#     # stored output from synthesis
#     # safe_filename = make_safe_filename(prompt)

#     stored_output_folder = "/home/jovyan/work/process/stored_outputs_pom_seed_128/"+safe_prompt+"/"

#     # stored_output_folder = "/home/jovyan/work/process/stored_outputs_pom_seed_128/"+\
#     #     "a_mushrooms_above_ground_growth_is_the_result_of_a_set_of_instructions_20210914_214118/"

#     interpbase = '/home/jovyan/work/interpolation/pom_interp_128_hold_{0}'
#     filebase = '/home/jovyan/work/interpolation/pom_128_%s_hold_{0}'

#     generate_video_hold(stored_output_folder, class_filenames, result_filename, prompt, interpbase, filebase)
#     print('scp "user@z8.local:/home/user/work/interpolation/pom_128_30_hold_{0}*" .'.format(safe_prompt))

Workaround to download files

In [ ]:
# prompts = [
#     "the sublime experience of an iceberg",
#     "ice is a vastness of possibilities",
#     "a vastness of spatial dimensions",
#     "a space of unlimited possibilities that the network must explore",
#     "a network",
#     "a network of vast spatial dimensions"
#     "eighteenth century painting of humans encountering nature",
#     "a person encountering nature",
#     "an elegant machine that learns to generate artificial images",
#     "a drawing of an elegant machine",
#     "a machine that learns to make images",
#     "a drawing of a machine that learns to make images",
# ]
# for prompt in prompts:
    
#     # prompt = "a silicon mold of a friends hands"

#     safe_prompt = make_safe_filename(prompt)
#     print('scp "user@z8.local:/home/user/work/interpolation/pom_128_30_hold_{0}*" .'.format(safe_prompt))

# Notes


Based on SIREN+CLIP Colabs by: [@advadnoun](https://twitter.com/advadnoun), [@norod78](https://twitter.com/norod78)

Other CLIP notebooks: [OpenAI tutorial](https://colab.research.google.com/github/openai/clip/blob/master/Interacting_with_CLIP.ipynb), [SIREN by @advadnoun](https://colab.research.google.com/drive/1FoHdqoqKntliaQKnMoNs3yn5EALqWtvP), [SIREN by @norod78](https://colab.research.google.com/drive/1K1vfpTEvAmxW2rnhAaALRVyis8EiLOnD), [BigGAN by @advadnoun](https://colab.research.google.com/drive/1NCceX2mbiKOSlAd_o7IU7nA9UskKN5WR), [BigGAN by @eyaler](j.mp/bigclip), [BigGAN by @tg_bomze](https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/Text2Image_v2.ipynb), [BigGAN using big-sleep library by @lucidrains](https://colab.research.google.com/drive/1MEWKbm-driRNF8PrU7ogS5o3se-ePyPb), [BigGAN story hallucinator by @bonkerfield](https://colab.research.google.com/drive/1jF8pyZ7uaNYbk9ZiVdxTOajkp8kbmkLK), [StyleGAN2-ADA Anime by @nagolinc](https://colab.research.google.com/github/nagolinc/notebooks/blob/main/TADNE_and_CLIP.ipynb) [v2](https://colab.research.google.com/github/nagolinc/notebooks/blob/main/CLIP_%2B_TADNE_(pytorch)_v2.ipynb)

Using the works:

https://github.com/openai/CLIP

https://tfhub.dev/deepmind/biggan-deep-512

https://github.com/huggingface/pytorch-pretrained-BigGAN

http://www.aiartonline.com/design-2019/eyal-gruss (WanderGAN)

For a curated list of more online generative tools see: [j.mp/generativetools](https://j.mp/generativetools)


# Leftovers

Ex2. Interpolating between two vectors:

In [ ]:
# np.random.RandomState(1)
# np.random.seed(1)
# torch.manual_seed(1)

# num_steps = 500

# # Prepare a input
# truncation = 1.0
# workbase = '/home/jovyan/'

# # sculptures: 


# # work/visual-imagination/CLIP/class_sculpture television buddha.txt
# # work/visual-imagination/CLIP/class_a buddha sculpture with television in the grass.txt
# # work/visual-imagination/CLIP/class_television buddha sculpture with grass.txt
# # work/visual-imagination/CLIP/class_old television in grass with buddha sculpture by pacific ocean.txt
# # work/visual-imagination/CLIP/class_old television in long grass with buddha sculpture by pacific ocean.txt


# noise1 = np.loadtxt(workbase+'work/output_20210222_195143/noise_00002.txt')
# class1 = np.loadtxt(workbase+'work/output_20210222_195143/class_00002.txt')

# noise2 = np.loadtxt(workbase+'work/output_20210222_195143/noise_00008.txt')
# class2 = np.loadtxt(workbase+'work/output_20210222_195143/class_00008.txt')

# # noise_vector = np.loadtxt(workbase+'work/visual-imagination/CLIP/noise_a room with good lighting.txt')
# # class_vector = np.loadtxt(workbase+'work/visual-imagination/CLIP/class_a room with good lighting.txt')

# # interpolate
# noises = interpolate_points(noise1, noise2, num_steps)
# classes = interpolate_points(class1, class2, num_steps)

# # expand dims (only necessary for single vector)

# for i in range(num_steps):
    
#     # expand dims
#     noise_vector = np.expand_dims(noises[i], axis=0)
#     class_vector = np.expand_dims(classes[i], axis=0)

#     # convert to tensors
#     noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
#     class_vector = torch.tensor(class_vector, dtype=torch.float32)

#     print(noise_vector.shape, noise_vector.dtype, class_vector.shape, class_vector.dtype)

#     # If you have a GPU, put everything on cuda
#     noise_vector = noise_vector.to('cuda')
#     noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
#     class_vector = class_vector.to('cuda')
#     class_vector = class_vector.softmax(dim=-1)
#     model.to('cuda')

#     # Generate an image
#     with torch.no_grad():
#         output = model(noise_vector, class_vector, truncation)

#     # If you have a GPU put back on CPU
#     output = output.to('cpu')

#     # # If you have a sixtel compatible terminal you can display the images in the terminal
#     # # (see https://github.com/saitoha/libsixel for details)
#     # # display_in_terminal(output)
#     save_as_images(output, workbase+"work/output_%05d" % i)
# #     clear_output()
# #     display(JupImage("output_%05d_0.png" % i))
#     i+=1

Ex.1: Generate from a single stored noise/class vector

In [ ]:
# # random seeding
# np.random.RandomState(1)
# np.random.seed(1)
# torch.manual_seed(1)


# # Prepare a input
# truncation = 1.0
# workbase = '/home/jovyan/'
# noise_vector = np.loadtxt(workbase+'work/results/That_never_left_any_alive_who_stayed_in_it._noise.txt')
# class_vector = np.loadtxt(workbase+'work/results/That_never_left_any_alive_who_stayed_in_it._class.txt')

# # noise_vector = np.loadtxt(workbase+'work/visual-imagination/CLIP/noise_a room with good lighting.txt')
# # class_vector = np.loadtxt(workbase+'work/visual-imagination/CLIP/class_a room with good lighting.txt')

# # noise_vector = noise_vectors
# # class_vector = class_vectors

# # expand dims
# noise_vector = np.expand_dims(noise_vector, axis=0)
# class_vector = np.expand_dims(class_vector, axis=0)

# # All in tensors
# noise_vector = torch.tensor(noise_vector, dtype=torch.float32)
# class_vector = torch.tensor(class_vector, dtype=torch.float32)

# print(noise_vector.shape, noise_vector.dtype, class_vector.shape, class_vector.dtype)

# # If you have a GPU, put everything on cuda
# noise_vector = noise_vector.to('cuda')
# noise_vector = noise_vector.clamp(-2*truncation, 2*truncation)
# class_vector = class_vector.to('cuda')
# class_vector = class_vector.softmax(dim=-1)
# model.to('cuda')

# # Generate an image
# with torch.no_grad():
#     output = model(noise_vector, class_vector, truncation)

# # If you have a GPU put back on CPU
# output = output.to('cpu')

# # # If you have a sixtel compatible terminal you can display the images in the terminal
# # # (see https://github.com/saitoha/libsixel for details)
# # # display_in_terminal(output)
# # save_as_images(output, "output_%s" % index)

# # display(Image("output_0.png"))

# # # Save results as png images
# save_as_images(output)

# Other Prompts

In [ ]:
# sculptures: 

# class_filenames = [
# "work/visual-imagination/CLIP/class_sculpture television buddha.txt",
# "work/visual-imagination/CLIP/class_a buddha sculpture with television in the grass.txt",
# "work/visual-imagination/CLIP/class_television buddha sculpture with grass.txt",
# "work/visual-imagination/CLIP/class_old television in grass with buddha sculpture by pacific ocean.txt",
# "work/visual-imagination/CLIP/class_old television in long grass with buddha sculpture by pacific ocean.txt"
# ]

# noise_filenames = [
# "work/visual-imagination/CLIP/noise_sculpture television buddha.txt",
# "work/visual-imagination/CLIP/noise_a buddha sculpture with television in the grass.txt",
# "work/visual-imagination/CLIP/noise_television buddha sculpture with grass.txt",
# "work/visual-imagination/CLIP/noise_old television in grass with buddha sculpture by pacific ocean.txt",
# "work/visual-imagination/CLIP/noise_old television in long grass with buddha sculpture by pacific ocean.txt"
# ]

# prompts = [
#     'a photo of wild tarragon',
#     'a drawing of wild tarragon, a tasteless planhttps://lilianweng.github.io/lil-log/2018/02/19/a-long-peek-into-reinforcement-learning.htmlt',
#     'a painting of farm hands, a kind of laborer',
#     'a painting of a farmer’s hands',
#     'a self-portrait of Artemisia Gentileschi, artist',
#     'artemisia Gentileschi is a dragon',
#     'a painting of Artemisia Gentileschi as a dragon',
#     'a photo of the dragon Artemisia Gentileschi',
#     'a portrait of artist as dragon',
#     'a drawing of a dragon',
#     'a painting of uprooted rhizome as a dragon',
#     'a sketch of a rhizome, uprooted',
#     'an image of a plant rising',
#     'a drawing of plant roots and mycorrhizal fungi',
#     'an image of growing wiser',
#     'a painting of wise plants',
#     'a drawing of plant wisdom',
#     'a photo of a plant hiding',
#     'a drawing of hiding from elders',
#     'a painting of Susanna and the Elders',
#     'an image of creeps',
#     'a painting of gazing creeps',
#     'a painting of groping creeps',
#     'a painting of invasive elders',
#     'a photo of perverse hope',
#     'a painting of your hatred',
#     'a drawing of killing a mosquito',
#     'a painting of a mosquito, a kind of corpse',
#     'a drawing of malaria',
#     'a sketch of salted fields',
#     'a photo of dancers',
#     'a painting of dancers in a field',
#     'an image of your spit',
#     'a photo of standing too close',
#     'a painting of someone standing too close',
#     'a drawing of an oak sapling',
#     'a painting of an oak in an empty field',
#     'a photo of growing',
#     'an image of growing wilder',
#     'a painting of growing stronger',
#     'a photo of a hand holding high',
#     'a painting of a hand holding the head of Holofernes',
#     'a painting of the head of Holofernes',
#     'a drawing of a head, blood-rooted',
#     'an image of a bloody root',
#     'a painting of autumn gold',
#     'a photo of a golden gown',
#     'an image of a mouth tasting',
#     'a sketch of a mouth',
#     'a drawing of taste',
#     'a painting of the taste of nothing',
#     'a photograph of being invisible',
#     'a drawing of your renown',
#     'a painting of a renowned artist',
#     'a portrait of the artist',
#     'a self-portrait of Artemisia Gentileschi as tarragon'
# ]

# prompts = [
#     'a picture of the interior of a bedroom',
#     'a drawing of the interior of a bedroom', 
#     'a photo of the interior of a bedroom', 
#     'a photo of a bathroom',
#     'a photo of a kitchen', 
#     'a photo of a front door'
# ]

# prompts = [
#     'a photo of a front door',
#     'a picture of the mud room',
#     'a photo of the kitchen', 
#     'a photo of the livingroom with television', 
#     'a photo of a couch and a television', 
#     'a photo of a family on a couch', 
#     'a picture a kitchen', 
#     'a photo of food on the kitchen counter',
#     'a photo of a pie in the oven',
#     'a photo of a bathroom',
#     'a photo of the interior of a bathroom',
#     'a photo of a person in a shower', 
#     'a photo of person brushing their teeth',
#     'a picture of the interior of a bedroom',
#     'a picture of a person sleeping in a bed', 
#     'a photo of a sunrise through a window'
# ]

# prompts = [
#     "sunrise through a window",
#     "a cat in the refrigerator"
# ]

# prompts = [
#     "over my head, I see the bronze butterfly",
#     "asleep on the black trunk",
#     "blowing like a leaf in green shadow",   
#     "down the ravine behind the empty house",   
#     "the cowbells follow one another",   
#     "into the distances of the afternoon",   
#     "to my right",
#     "in a field of sunlight between two pines",   
#     "the droppings of last year’s horses",   
#     "blaze up into golden stones",
#     "I lean back, as the evening darkens and comes on",
#     "a chicken hawk floats over, looking for home",
#     "I have wasted my life"
# ]

# prompts = [
#     "Midway on our lifes journey, I found myself",
#     "In dark woods, the right road lost",
#     "To tell About those woods is hard - so tangled and rough",
#     "And savage that thinking of it now, I feel",
#     "The old fear stirring: death is hardly more bitter.",
#     "And yet, to treat the good I found there as well",
#     "I'll tell what I saw, though how I came to enter",
#     "I cannot well say, being so full of sleep",
#     "Whatever moment it was I began to blunder",
#     "Off the true path. But when I came to stop",
#     "Below a hill that marked one end of the valley",
#     "That had pierced my heart with terror, I looked up",
#     "Toward the crest and saw its shoulders already",
#     "Mantled in rays of that bright planet that shows",
#     "The road to everyone, whatever our journey.",
#     "Then I could feel the terror begin to ease",
#     "That churned in my heart's lake all through the night.",
#     "As one still panting, ashore from dangerous seas",
#     "Looks back at the deep he has escaped, my thought",
#     "Returned, still fleeing, to regard that grim defile", 
#     "That never left any alive who stayed in it."
# ]
